<a href="https://colab.research.google.com/github/thdefn/jango/blob/master/saving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from tensorflow import keras

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization 
from tensorflow.keras.utils import to_categorical

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt

import seaborn as sns
import os
import glob
import plotly.graph_objects as go
import cv2
from PIL import Image
from PIL import ImageFile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

from tqdm import tqdm

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/My Drive/data/.kaggle

/content/gdrive/My Drive/data/.kaggle


In [ ]:
root_dir = './'
train_dir = os.path.join(root_dir,'train')

type1_dir = os.path.join(train_dir, 'Type_1')
type2_dir = os.path.join(train_dir, 'Type_2')
type3_dir = os.path.join(train_dir, 'Type_3')

train_type1_files = glob.glob(type1_dir+'/*.jpg')
train_type2_files = glob.glob(type2_dir+'/*.jpg')
train_type3_files = glob.glob(type3_dir+'/*.jpg')


added_type1_files  =  glob.glob(os.path.join(root_dir, "additional_Type_1_v2", "Type_1")+'/*.jpg')
added_type2_files  =  glob.glob(os.path.join(root_dir, "additional_Type_2_v2", "Type_2")+'/*.jpg')
added_type3_files  =  glob.glob(os.path.join(root_dir, "additional_Type_3_v2", "Type_3")+'/*.jpg')


type1_files = train_type1_files + added_type1_files
type2_files = train_type2_files + added_type2_files
type3_files = train_type3_files + added_type3_files

print(f'''Type 1 files for training: {len(type1_files)} 
Type 2 files for training: {len(type2_files)}
Type 3 files for training: {len(type3_files)}''' )

Type 1 files for training: 1440 
Type 2 files for training: 4346
Type 3 files for training: 2426


In [ ]:
files = {'filepath': type1_files + type2_files + type3_files,
          'label': ['Type 1']* len(type1_files) + ['Type 2']* len(type2_files) + ['Type 3']* len(type3_files)}

files_df = pd.DataFrame(files).sample(frac=1, random_state= 1).reset_index(drop=True)
files_df

,filepath,label
0,./train/Type_1/1414.jpg,Type 1
1,./additional_Type_2_v2/Type_2/824.jpg,Type 2
2,./train/Type_2/834.jpg,Type 2
3,./additional_Type_2_v2/Type_2/5091.jpg,Type 2
4,./additional_Type_3_v2/Type_3/1264.jpg,Type 3
...,...,...
8207,./additional_Type_2_v2/Type_2/6709.jpg,Type 2
8208,./additional_Type_3_v2/Type_3/3543.jpg,Type 3
8209,./additional_Type_1_v2/Type_1/5406.jpg,Type 1
8210,./additional_Type_2_v2/Type_2/3502.jpg,Type 2


In [ ]:
t = ImageDataGenerator(rescale=1./ 255, 
                       rotation_range = 40,
                        zoom_range = 0.2,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        horizontal_flip = True,
                        vertical_flip = True)

t_gen = t.flow_from_dataframe(files_df,x_col='filepath', y_col='label', target_size = (128, 128),batch_size=16,class_mode='categorical', color_mode='rgb', subset='training')

Found 8212 validated image filenames belonging to 3 classes.


In [ ]:
#train_df, valid_df = train_test_split(files_df, test_size = 0.2, stratify = files_df['label'], random_state = 1234)
k = ImageDataGenerator(rescale=1./ 255, validation_split=0.2)
t_nongen = k.flow_from_dataframe(files_df, x_col='filepath', y_col='label', target_size = (128, 128), class_mode='categorical', color_mode='rgb', subset='training')
v_nongen = k.flow_from_dataframe(files_df, x_col='filepath', y_col='label', target_size = (128, 128), class_mode='categorical', color_mode='rgb', subset='validation')

Found 6570 validated image filenames belonging to 3 classes.
Found 1642 validated image filenames belonging to 3 classes.


In [ ]:
'''# 사전 교육된 기본 모델 가져오기
base_model = VGG19(include_top=False, weights='imagenet',input_shape=(128, 128, 3))

base_model.trainable = False

# 교육할 계층 설정 최상위 계층만 교육하기 위해 (False 가중치 고정)
for layer in base_model.layers:
    layer.trainable = False
    
# 기본 모델의 마지막 층
x = base_model.output

x = Flatten()(x)
h1 = Dense(512, activation='relu')(x)
drop1 = Dropout(rate=0.5)(h1)
h2 = Dense(512, activation='relu')(drop1)
drop2 = Dropout(rate=0.5)(h2)
output = Dense(3, activation='softmax')(drop2)

trs = Model(inputs=base_model.input, outputs=output)
trs.summary()'''

"# 사전 교육된 기본 모델 가져오기\nbase_model = VGG19(include_top=False, weights='imagenet',input_shape=(128, 128, 3))\n\nbase_model.trainable = False\n\n# 교육할 계층 설정 최상위 계층만 교육하기 위해 (False 가중치 고정)\nfor layer in base_model.layers:\n    layer.trainable = False\n    \n# 기본 모델의 마지막 층\nx = base_model.output\n\nx = Flatten()(x)\nh1 = Dense(512, activation='relu')(x)\ndrop1 = Dropout(rate=0.5)(h1)\nh2 = Dense(512, activation='relu')(drop1)\ndrop2 = Dropout(rate=0.5)(h2)\noutput = Dense(3, activation='softmax')(drop2)\n\ntrs = Model(inputs=base_model.input, outputs=output)\ntrs.summary()"

### 세이빙한 모델 가져오는 부분

In [ ]:
trs = keras.models.load_model('my_model.h5') #모델 가져오기
trs.compile(optimizer=RMSprop(lr=1e-4),
  loss='categorical_crossentropy', 
  metrics=['accuracy']) #옵티마이저 컴파일

In [ ]:
general = trs.fit(t_nongen, validation_data= v_nongen, epochs= 10, verbose=1) #피팅
trs.save('my_model2.h5') #모델 세이브

Epoch 1/10
206/206 [==============================] - 6104s 30s/step - loss: 0.8373 - accuracy: 0.6027 - val_loss: 0.7915 - val_accuracy: 0.6413
Epoch 2/10
201/206 [============================>.] - ETA: 1:54 - loss: 0.8265 - accuracy: 0.6087

In [ ]:
general.history

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(general.history['accuracy'], label='acc')
plt.plot(general.history['var_accuracy'], label='var_acc')
plt.legend(loc='lower right')
plt.title('Accuracy comparison')

plt.subplot(1, 2, 2)
plt.plot(general.history['loss'], label='loss')
plt.plot(general.history['var_loss'], label='var_loss')
plt.legend(loc='upper right')
plt.title('Loss comparison')

plt.show()

In [ ]:
'''trs2 = Model(inputs=base_model.input, outputs=output)
trs2.compile(optimizer=RMSprop(lr=1e-4),
  loss='categorical_crossentropy',
  metrics=['accuracy'])
verimggen = trs2.fit(t_gen, steps_per_epoch = t_gen.samples//16, validation_data= v_nongen, epochs= 100, verbose=1)
trs2.save('my_model.h5')'''